In [1]:
import torch
torch.cuda.set_device(2)
print(torch.cuda.current_device())

2


In [2]:
%run ../prepare_data.py

In [3]:
batch_size = 32

In [4]:
train_loader, valid_loader, valid_dataset = rsna_dataloaders(batch_size)

In [5]:
x, y = next(iter(train_loader))

In [6]:
x.shape, y.shape

(torch.Size([32, 3, 256, 256]), torch.Size([32]))

### Full model

In [7]:
def basicblock(in_, 
               out_, 
               kernel_size=3, 
               stride=1, 
               groups=1):

    
    padding = (kernel_size - 1) // 2
    block = nn.Sequential(nn.Conv2d(in_, out_, kernel_size, stride, padding, groups=groups, bias=False),
                          nn.BatchNorm2d(out_),
                          nn.ReLU6(inplace=True))
    
    return block

class InvertedResblock(nn.Module):
    
    def __init__(self, in_, out_, kernel_size=3, stride=1, expand_ratio=1):
        super(InvertedResblock, self).__init__()
        
        self.stride = stride
        hidden_dim = int(round(in_ * expand_ratio))
        blocks = []
        self.use_res_connect = self.stride == 1 and in_ == out_
        
        if expand_ratio != 1:
            blocks.append(basicblock(in_, hidden_dim, kernel_size=1))

        blocks.append(basicblock(hidden_dim, hidden_dim, kernel_size=kernel_size, stride=stride, groups=hidden_dim))
        blocks.append(nn.Conv2d(hidden_dim, out_, kernel_size=1, stride=1, bias=False))
        blocks.append(nn.BatchNorm2d(out_))

        self.blocks = nn.Sequential(*blocks)
    
    def forward(self, x):
        if self.use_res_connect:
            return x + self.blocks(x)
        else:
            return self.blocks(x)

In [8]:
class Net(nn.Module):
    def __init__(self, num_classes=1):
        super(Net, self).__init__()
        self.features1 = nn.Sequential(basicblock(3, 32, 3, 2, 1),
                                       InvertedResblock(32, 16, 3, 1, 1), 
                                       InvertedResblock(16, 24, 3, 2, 6),
                                       InvertedResblock(24, 24, 3, 1, 6),
                                       InvertedResblock(24, 32, 3, 2, 6),
                                       InvertedResblock(32, 32, 3, 1, 6),
                                       InvertedResblock(32, 32, 3, 1, 6), 
                                       InvertedResblock(32, 64, 3, 2, 6),
                                       InvertedResblock(64, 64, 3, 1, 6),
                                       InvertedResblock(64, 64, 3, 1, 6),
                                       InvertedResblock(64, 64, 3, 1, 6),
                                       InvertedResblock(64, 96, 3, 1, 6),
                                       InvertedResblock(96, 96, 3, 1, 6),
                                       InvertedResblock(96, 96, 3, 1, 6),
                                       InvertedResblock(96, 160, 3, 2, 6))
    
        self.features2 = nn.Sequential(InvertedResblock(160, 160, 3, 1, 6),
                                       InvertedResblock(160, 160, 3, 1, 6),
                                       InvertedResblock(160, 320, 3, 1, 6),
                                       basicblock(320, 1280, 1))
        self.classifier = nn.Sequential(nn.Dropout(p=0.2), nn.Linear(1280, 1000))
        
    def forward(self, x):
        x = self.features1(x)
        x = self.features2(x)
        x = x.mean(3).mean(2)
        x = self.classifier(x)

        return x

In [9]:
model = Net().cuda()

In [10]:
sum(p.numel() for p in model.parameters())

3504872

In [11]:
load_model(model, '/home/rimmanni/imagenet/mobilenet_320_iter20_revised.pth')

In [12]:
class Net2(nn.Module):
    def __init__(self, model):
        super(Net2, self).__init__()
        self.features1 = model.features1
        self.features2 = model.features2
        self.classifier = nn.Sequential(nn.Dropout(p=0.2), nn.Linear(1280, 1))
        
    def forward(self, x):
        x = self.features1(x)
        x = self.features2(x)
        x = x.mean(3).mean(2)
        x1 = self.classifier(x).view(-1)

        return x1

In [13]:
model2 = Net2(model).cuda()

In [ ]:
data = []
for i in range(5):
    model = Net().cuda()
    load_model(model, '/home/rimmanni/imagenet/mobilenet_320_iter20_revised.pth')
    model2 = Net2(model).cuda()
    optimizer = create_optimizer(model2, 0.01)
    score, t = train_triangular_policy(model2, optimizer, train_loader, valid_loader, valid_dataset,
                                               loss_fn=F.binary_cross_entropy_with_logits, 
                                               dataset='rsna', binary=True, max_lr=0.01, epochs=15)
    
    data.append([score, t, 'rsna', 'mobilenet_full'])
    

train_loss 0.369 val_loss 0.337 val_auc_score 0.915
----End of step 0:01:45.098232
train_loss 0.318 val_loss 0.306 val_auc_score 0.932
----End of step 0:01:41.192611
train_loss 0.304 val_loss 0.337 val_auc_score 0.942
----End of step 0:01:42.937131
train_loss 0.292 val_loss 0.284 val_auc_score 0.944
----End of step 0:01:42.036379
train_loss 0.276 val_loss 0.292 val_auc_score 0.942
----End of step 0:01:44.143949
train_loss 0.273 val_loss 0.272 val_auc_score 0.948
----End of step 0:01:45.091344


### other

In [ ]:
%run ../architectures.py

#### 0.25_1

In [ ]:
for i in range(5):
    model = MobileNet(num_classes=1000, width_mult=0.25, depth_mult=1.0).cuda()
    load_model(model, '/home/rimmanni/imagenet/mobilenet_0_25_1_iter20.pth')
    model.classifier = nn.Sequential(nn.Dropout(p=0.2), nn.Linear(1280, 1)).cuda()
    optimizer = create_optimizer(model, 0.01)
    score, t = train_triangular_policy(model, optimizer, train_loader, valid_loader, valid_dataset,
                                       loss_fn=F.binary_cross_entropy_with_logits, 
                                       dataset='rsna', binary=True, max_lr=0.01, epochs=15)
    
    data.append([score, t, 'rsna', 'mobilenet_0_25_1'])
    

#### 0_5_0_5

In [ ]:
for i in range(5):
    model = MobileNet(num_classes=1000, width_mult=0.5, depth_mult=0.5).cuda()
    load_model(model, '/home/rimmanni/imagenet/mobilenet_0_5_0_5_iter20.pth')
    model.classifier = nn.Sequential(nn.Dropout(p=0.2), nn.Linear(1280, 1)).cuda()
    optimizer = create_optimizer(model, 0.01)
    score, t = train_triangular_policy(model, optimizer, train_loader, valid_loader, valid_dataset,
                                       loss_fn=F.binary_cross_entropy_with_logits, 
                                       dataset='rsna', binary=True, max_lr=0.01, epochs=15)
    
    data.append([score, t, 'rsna', 'mobilenet_0_5_0_5'])
    

#### 0_75_0_3

In [ ]:
for i in range(5):
    model = MobileNet(num_classes=1000, width_mult=0.75, depth_mult=0.3).cuda()
    load_model(model, '/home/rimmanni/imagenet/mobilenet_0_75_0_3_iter20.pth')
    model.classifier = nn.Sequential(nn.Dropout(p=0.2), nn.Linear(1280, 1)).cuda()
    optimizer = create_optimizer(model, 0.01)
    score, t = train_triangular_policy(model, optimizer, train_loader, valid_loader, valid_dataset,
                                       loss_fn=F.binary_cross_entropy_with_logits, 
                                       dataset='rsna', binary=True, max_lr=0.01, epochs=15)
    
    data.append([score, t, 'rsna', 'mobilenet_0_75_0_3'])
    

#### 1_0_2

In [ ]:
for i in range(5):
    model = MobileNet(num_classes=1000, width_mult=1.0, depth_mult=0.2).cuda()
    load_model(model, '/home/rimmanni/imagenet/mobilenet_1_0_2_iter20.pth')
    model.classifier = nn.Sequential(nn.Dropout(p=0.2), nn.Linear(1280, 1)).cuda()
    optimizer = create_optimizer(model, 0.01)
    score, t = train_triangular_policy(model, optimizer, train_loader, valid_loader, valid_dataset,
                                       loss_fn=F.binary_cross_entropy_with_logits, 
                                       dataset='rsna', binary=True, max_lr=0.01, epochs=15)
    
    data.append([score, t, 'rsna', 'mobilenet_1_0_2'])
    

In [ ]:
columns = ['score', 'time', 'dataset', 'model']
df = pd.DataFrame(data=data, columns=columns)

In [ ]:
df.to_csv("rsna_mobilenet.csv", index=False)